# Training und Anwendung eines YoloV4-Detektors

Dieses Tutorial basiert auf https://github.com/theAIGuysCode/YOLOv4-Cloud-Tutorial/blob/master/YOLOv4_Training_Tutorial.ipynb und wurde von uns gekürzt sowie auf deutsch übersetzt.

# Schritt 1: Hardware-Beschleunigung aktivieren


Google Colaboratory unterstützt die Verwendung von Google eigenen GPUs wie der NVIDIA Tesla K80. Dies ermöglicht ein schnelleres Training sowie schnellere Detektion im Vergleich zur alleinigen Nutzung der CPU.

Um die GPU-Unterstuützung zu aktivieren: Edit/Bearbeiten -> Notebook Settings/Notebook Einstellungen und dort den Hardware Accelerator/Hardware Beschleuniger auf GPU einstellen.


# Schritt 2: Klonen von Darknet

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# Aktivieren von CUDA, GPU und OpenCV
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# Verifizieren, dass CUDA aktiviert ist
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# make Befehl für Darknet, Warnungen können hierbei ignoriert werden
!make

# Schritt 3 (optional)

Im GitHub Repository von AlexeyAB findet sich bereits ein vortrainiertes Modell mit dem über 80 Klassen erkannt werden. Wir können das hier nutzen, um zu verifizieren, dass Darknet ordnungsgemäß funktioniert.

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

Damit innerhalb von Google Colaboratory Bilder (und deren Overlays) angezeigt werden können, muss man noch eine Hilfsfunktion definieren.

In [ ]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

Darknet beinhaltet einige Bilder zum Testen. Diese finden sich unter `darknet/data/`

In [ ]:
# testet Darknet auf einem Beispielbild
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/person.jpg

Nun kann man die vordefinierte Funktion für die Ausgabe nutzen:

In [ ]:
imShow('predictions.jpg')

# Schritt 4: Google Drive verbinden

Es gibt verschiedene Möglichkeiten, die erforderlichen Dateien (Bilder, Config-Dateien, Label-Dateien, etc.) in Colabs zu importieren. Wir haben hier die Anbindung zu Google Drive genutzt, da dies uns u.A. eine automatische Backup-Funktionalität liefert.

In [ ]:
# Dieser Code bindet Google Drive an Colabs an. Hierfür wird ein Bestätigungscode angefordert.
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# erstellt einen symbolischen Link zum Zugriff auf die Daten in Google Drive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Testen der Konfiguration (optional):
Hier einfach den entsprechenden Path eintragen

In [ ]:
%cd darknet
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights /mydrive/../image.jpg
imShow('predictions.jpg')

# Schritt 5: Ordnerstruktur und Konfiguration
Im Root-Verzeichnis von Google Drive haben wir einen Ordner "yolov4" angelegt. Dieser enthält den Unterordner "backup", in dem später die Zwischenergebnisse abgelegt werden.



In [ ]:
# hiermit kann man sich den Inhalt des Ordners zur Kontrolle anzeigen lassen
!ls /mydrive/yolov4

## Schritt 5.1: Bildmaterial

In [ ]:
# nun kopieren wir die zip-Archive der Trainigs- und Testbilder von Google Drive in Colabs
!cp /mydrive/yolov4/food_train.zip ../
!cp /mydrive/yolov4/food_validate.zip ../

In [ ]:
# ... und entpacken diese anschließend (je nach Anzahl der Bilder können kopieren und entpacken einige Zeit in Anspruch nehmen)
!unzip ../food_train.zip -d data/
!unzip ../food_validate.zip -d data/

## Schritt 5.2 Cfg-Datei
In dieser Datei finden einige Konfigurationen zum Training statt. Wir haben uns dabei am Tutorial orientiert.

In [ ]:
# die Template Datei wird in Drive kopiert und kann dort bearbeitet werden. Entsprechende Dateinamen bitte ändern.
!cp cfg/yolov4-custom.cfg /mydrive/yolov4/yolov4-food.cfg

Gemäß des Tutorials ergeben sich folgende Konfiguration:

*   batch = 64
*   subdivision = 16 (alternativ 32)
*   width = 416
*   height = 416 (für Höhe und Breite können Vielfache von 32 genutzt werden. Je höher der Wert, desto akkurater das Ergebnis, aber desto langsamer das Training)
*   max_batches = (Anzahl d. Klassen) * 2000 (mindestens aber 6000)
*   steps = (0.8 * max_batches), (0.9 * max_batches)
*   filters (Anzahl d. Klassen + 5) * 3
*   **Beachte: Die Werte müssen u.U. mehrfach gesetzt werden.**
*   *Optional: random = 1 zu random = 0, falls Speicherprobleme auftreten oder das Training ungewöhnlich lange dauert*


In [ ]:
# Upload der modifizieren Datei aus Drive
!cp /mydrive/yolov4/[...].cfg ./cfg

## Schritt 5.3 Names- und Data-Datei
Erstelle eine .data Datei mit folgendem Inhalt:

```
classes = (number of classes)
train = data/train.txt
valid = data/test.txt
names = data/[...].names 
backup = /mydrive/yolov4/backup
```



Die .names Datei ergibt sich aus der classes.txt Datei, die wird beim Labeln erstellt haben. Hierfür also einfach eine neue Datei erstellen, die Inhalte wieder zeilenweise einfügen und mit der Dateiendung .names abspeichern.

In [ ]:
# nun beide Dateien importieren
!cp /mydrive/yolov4/food.names ./data
!cp /mydrive/yolov4/food.data  ./data

## Schritt 5.4: train.txt und test.txt generieren

Die erforderlichen Skripte können hier bezogen werden: https://github.com/theAIGuysCode/YOLOv4-Cloud-Tutorial.
Die Dateien "generate_train.py" und "generate_test.py" dann einfach in den yolov4-Ordner kopieren.

In [ ]:
# Skripte ausführen
!python generate_train.py
!python generate_test.py

Im data/ Ordner von Colabs sollten nun die zwei Dateien "train.txt" und "test.txt" zu finden sein.

# Schritt 6: Training

## Schritt 6.1: Download von vortrainierten Gewichten für die Convolutional Layer (optional)
Diese Gewichte müssen nicht genutzt werden, erlauben aber ein effektiveres und schnelleres Training.

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

## Schritt 6.2: Custom Detector

In [ ]:
# hier nun die Namen der .data und .cfg Dateien eintragen. %%capture auskommentieren, wenn Speicherprobleme auftreten
#%%capture
!./darknet detector train data/[...].data cfg/[...].cfg yolov4.conv.137 -dont_show -map

Die Zwischenergebnisse des Trainings werden im backup Ordner abgelegt. Das erlaubt, das Training zu einem späteren Zeitpunkt von diesem Ergebnis aus fortzusetzen:

In [ ]:
!./darknet detector train data/[...].data cfg/[...].cfg /mydrive/yolov4/backup/[...].weights -dont_show

Man kann sich eine Übersicht über die Average Precision in Abhängigkeit der Anzahl der Iterationen anschauen:

In [ ]:
imShow('chart.png')

Um die Average Precision einzeln aufzuschlüssen, kann man folgenden Befehl verwenden:

In [ ]:
!./darknet detector map data/[...].data cfg/[...].cfg /mydrive/yolov4/backup/[...].weights

Um das trainierte Model nun mit einem fremden Bild zu testen, kann man folgenden machen:

In [ ]:
# Test Modus für cfg-Datei aktivieren
%cd cfg
!sed -i 's/batch=64/batch=1/' [...].cfg
!sed -i 's/subdivisions=16/subdivisions=1/' [...].cfg
%cd ..

In [ ]:
!./darknet detector test data/[...].data cfg/[...].cfg /mydrive/yolov4/backup/[...].weights /mydrive/images/[...].jpg -thresh 0.3
imShow('predictions.jpg')

# Geschafft

Es wurde ein neuer yolov4-Object-Detector trainiert. Dieser kann nun z.B. in ein Backend eingebunden werden.